In [18]:
__version__ = '0.1.0'

import torch.nn as nn
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification
import pandas as pd
import torch
from transformers import AutoTokenizer
from torch.utils.data import Dataset

class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.bert = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
        # Also include followers and following
        self.layer2 = nn.Linear(2, 1, bias=True)
        
    def forward(self, x):
        out = torch.zeros((x.shape[0], 2))

        bert_out = self.bert(x[:, :512].long(), x[:, 512:1024].long()).to_tuple()[0]
        out[:, :2] = bert_out
        res = self.layer2(out)
        return torch.relu(res)
model = Classifier()

model.load_state_dict(torch.load("model(2).bin"))


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        

def encode_sentence(sentence):
    res = torch.zeros((1, 1024))
    encoded = tokenizer(sentence, max_length=512, padding="max_length", truncation=True)
    res[0, :512] = torch.tensor(encoded['input_ids'])
    res[0, 512:1024] = torch.tensor(encoded['attention_mask'])
    return res

merged_file = "../data/merged_troll_data.json"

merged_df = pd.read_json(merged_file)

# while True:
#     encoded = encode_sentence(input("Please enter a tweet: "))
#     print(model(encoded).item())

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [19]:
confusion_matrix = np.zeros((2,2))

for troll in range(2):
    for i in range(2000):
        sentence = merged_df[merged_df["troll"] == bool(troll)].iloc[i].content
        encoded = encode_sentence(sentence)
        output = model(encoded).item()
        if output >= 0.5:
            confusion_matrix[troll,1] += 1
        else:
            # wrong answer
            confusion_matrix[troll,0] += 1
        if i % 50 == 0:
            print(f"{troll} -> {i}")
print(confusion_matrix)

0 -> 0
0 -> 50
0 -> 100
0 -> 150
0 -> 200
0 -> 250
0 -> 300
0 -> 350
0 -> 400
0 -> 450
0 -> 500
0 -> 550
0 -> 600
0 -> 650
0 -> 700
0 -> 750
0 -> 800
0 -> 850
0 -> 900
0 -> 950
0 -> 1000
0 -> 1050
0 -> 1100
0 -> 1150
0 -> 1200
0 -> 1250
0 -> 1300
0 -> 1350
0 -> 1400
0 -> 1450
0 -> 1500
0 -> 1550
0 -> 1600
0 -> 1650
0 -> 1700
0 -> 1750
0 -> 1800
0 -> 1850
0 -> 1900
0 -> 1950
1 -> 0
1 -> 50
1 -> 100
1 -> 150
1 -> 200
1 -> 250
1 -> 300
1 -> 350
1 -> 400
1 -> 450
1 -> 500
1 -> 550
1 -> 600
1 -> 650
1 -> 700
1 -> 750
1 -> 800
1 -> 850
1 -> 900
1 -> 950
1 -> 1000
1 -> 1050
1 -> 1100
1 -> 1150
1 -> 1200
1 -> 1250
1 -> 1300
1 -> 1350
1 -> 1400
1 -> 1450
1 -> 1500
1 -> 1550
1 -> 1600
1 -> 1650
1 -> 1700
1 -> 1750
1 -> 1800
1 -> 1850
1 -> 1900
1 -> 1950
[[1789.  211.]
 [ 730. 1270.]]
